In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

#from tensorflow.keras import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
#from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from keras import Sequential
from keras.optimizers import Adam
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [3]:
import cv2
from PIL import Image

In [4]:
df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/train.csv")
df_test = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/test.csv")

In [5]:
df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [6]:
# True image size
IMG_WIDTH = 120
IMG_HEIGHT = 80

def preprocess_image(image, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    
    :param img: A NumPy Array that will be cropped
    :param sigmaX: Value used for add GaussianBlur to the image
    
    :return: A NumPy array containing the preprocessed image
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image

In [7]:
y = df["Class"]

In [8]:
from matplotlib import pyplot as plt
%matplotlib inline

In [9]:
#img = cv2.imread(r'E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/image7042.jpg')
#cv2.imshow('image',img)
#cv2.waitKey(0)

In [10]:
# MAKING SURE ALL IMAGES ARE OF SAME SIZE

#diff_shape = set()
PATH = "E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/"
#for i in range(df.shape[0]):
#    temp = PATH + str(df["Image"][i])
#    img = cv2.imread(temp)
#    diff_shape.add(img.shape)

In [11]:
BATCH_SIZE = 16
# resizing the image to the size of efficientnet
IMG_WIDTH = 299
IMG_HEIGHT = 299

# Add Image augmentation to our generator
train_datagen = ImageDataGenerator(rotation_range=90,
                                   brightness_range=[0.2,1.0],
                                   zoom_range=[0.5,1.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.1,
                                   preprocessing_function=preprocess_image, 
                                   rescale=1 / 128.)
train_generator = train_datagen.flow_from_dataframe(df, 
                                                    x_col='Image', 
                                                    y_col='Class',
                                                    directory = PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical', 
                                                    subset='training')
val_generator = train_datagen.flow_from_dataframe(df, 
                                                  x_col='Image', 
                                                  y_col='Class',
                                                  directory = PATH,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  subset='validation')

Found 5385 validated image filenames belonging to 4 classes.
Found 598 validated image filenames belonging to 4 classes.


In [12]:
#Exception Net
from keras.applications.xception import Xception

exc = Xception(input_shape=(299,299,3),include_top=False, weights='imagenet')

In [13]:
exc.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [14]:
for i in exc.layers:
    i.trainable = False

In [15]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(r'E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/efficientnetB3_Pure.h5', monitor='categorical_crossentropy', mode='min', verbose=1, save_best_only=True)

In [ ]:
class testing():
    self.new_var = 5
    
    def print_n(self):
        print(new_var)
temp = testing()
temp.print_n()

In [68]:
# custom callback
class save_model_1(tf.keras.callbacks.Callback):
    def __init__(self,filepath):
        self.paths = filepath
    
    min_loss = 10000000

    def on_epoch_end(self, epoch, logs=None):
        print(logs['loss'])
        print(self.paths)
        if logs["loss"] < self.min_loss:
            min_loss = logs["loss"]
            temp_model = self.model
            mem = []
            self.model.save(self.paths)
            #for i in temp_model.layers:
            #    mem.append(i.trainable)
            #    i.trainable = False
            #
            #for c,i in enumerate(temp_model):
            #    i.trainable = mem[c]
        
save_model = save_model_1("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Models/timepass.h5")
#print(save_model.paths)
print(type(save_model))

<class '__main__.save_model_1'>


In [69]:
def build_model():
    """
    A custom implementation of EfficientNetB5
    for the APTOS 2019 competition
    (Regression)
    """
    model = Sequential()
    
    model.add(exc)
    
    model.add(GlobalAveragePooling2D())
    model.add(Dense(400, activation="elu"))
    model.add(Dense(200, activation="elu"))
    model.add(Dense(4, activation="softmax"))
    model.compile(loss='categorical_crossentropy',
                  optimizer="adam", 
                  metrics=['categorical_crossentropy', 'mse'])
    print(model.summary())
    return model

# Initialize model
model = build_model()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
global_average_pooling2d_11  (None, 2048)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 400)               819600    
_________________________________________________________________
dense_32 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_33 (Dense)             (None, 4)                 804       
Total params: 21,762,084
Trainable params: 900,604
Non-trainable params: 20,861,480
_________________________________________________________________
None


In [70]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=12)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=5, 
                        verbose=1, 
                        mode='auto',
                        epsilon=0.00001)

# Begin training
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=90,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // BATCH_SIZE,
                    callbacks=[mc,rlr])


C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/90
336/336 [==============================] - 124s 370ms/step - loss: 1.2503 - categorical_crossentropy: 1.2503 - mse: 0.1681 - val_loss: 1.1538 - val_categorical_crossentropy: 1.2330 - val_mse: 0.1674
1.2502999925418015
E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Models/timepass.h5
Epoch 2/90
 55/336 [===>..........................] - ETA: 1:34 - loss: 1.1431 - categorical_crossentropy: 1.1431 - mse: 0.1549

KeyboardInterrupt: 

In [ ]:
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    
    :param model: A Keras model object
    :param generator: A Keras ImageDataGenerator object
    
    :return: A tuple with two Numpy Arrays. One containing the predictions
    and one containing the labels
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / BATCH_SIZE))):
        print(_)
        x, y = next(generator)
        print(x,y)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()

In [ ]:
df_test.head(10)

In [ ]:
test_path = "E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Test Images/"

In [ ]:
df_test['Class'] = np.zeros(df_test.shape[0])
df_test["Class"] = df_test["Class"].apply(str)
df_test["Class"] = df_test["Class"].astype("category")
# For preprocessing test images
test_generator = ImageDataGenerator(preprocessing_function=preprocess_image, 
                                    rescale=1 / 128.).flow_from_dataframe(df_test,
                                                                          x_col = "Image",
                                                                          #y_col = "Class",
                                                                          directory=test_path,
                                                                          target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                                          batch_size=1,
                                                                          class_mode=None,
                                                                          shuffle=False)
filename = test_generator.filenames
number_test = len(filename)

In [ ]:
from keras.models import load_model

In [ ]:
best_model = load_model('E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/efficientnetB3_no_freezing.h5')

In [ ]:
predict = best_model.predict_generator(test_generator,number_test)

In [ ]:
def convert_pred(pred):
    output = []
    output1 = []
    for i in pred:
        temp = max(i)
        for c,j in enumerate(i):
            if j == temp:
                output1.append(c)
    for i in output1:
        if i==2:
            output.append("Food")
        if i==3:
            output.append("misc")
        if i==0:
            output.append("Attire")
        if i==1:
            output.append("Decorationandsignage")
    return output1,output

In [ ]:
evaluation,submission, = convert_pred(predict)

In [ ]:
submission

In [ ]:
evaluation

In [ ]:
df_test = df_test.drop("Class",axis =1)
df_test["Class"] = submission

In [ ]:
df_test.to_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/submission.csv",index = False)

In [ ]:
'''
for i in range(df_test.shape[0]):
    temp = test_path + str(df_test["Image"][i])
    print(df_test["Class"][i])
    img = cv2.imread(temp)
    cv2.imshow('image',img)
    cv2.waitKey(0)
'''